In [1]:
import numpy as np
import pandas as pd

# 필요한 Surprise 알고리즘 불러오기
from surprise import BaselineOnly 
from surprise import KNNWithMeans
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

In [26]:
# csv 파일에서 불러오기
df = pd.read_csv("../../train_dataset/personalRecsys/preference.csv")
df_card  = pd.read_csv("../../train_dataset/personalRecsys/card_info.csv", sep=r',(?!\s)', warn_bad_lines=False, error_bad_lines=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


### 학습데이터에서 공지사항 필터링

In [3]:
df2 = df[df['score'] == 1]
df3 = df[df['score'] != 1]

df2_1 = df2.drop(df2.index[0:400000])

ratings = pd.concat([df2_1, df3])
ratings.reset_index(drop=True, inplace=True)
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217623 entries, 0 to 217622
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   member_no  217623 non-null  int64
 1   card_no    217623 non-null  int64
 2   score      217623 non-null  int64
dtypes: int64(3)
memory usage: 5.0 MB


In [4]:
ratings.loc[ratings['score'] >= 5, 'score'] = 5

In [5]:
ratings.sort_values(by=['score'], ascending=False).drop_duplicates()

,member_no,card_no,score
209863,137334159,6761872,5
165069,42131003,6764957,5
183389,93542003,6766315,5
154702,150471559,6875325,5
154703,150536026,6792874,5
...,...,...,...
72544,124909900,6839756,1
72545,125075189,6831585,1
72546,125402312,6698870,1
72547,125494430,6774147,1


In [6]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(ratings[['member_no', 'card_no', 'score']], reader)

In [7]:
# Train/Test 분리 
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

In [8]:
# 정확도 계산 
#sim_options = {'name': 'pearson_baseline', 'user_based': True}
#algo = KNNWithMeans(k=30, sim_options=sim_options)
algo = SVD(n_factors=50, n_epochs=20, random_state=42)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.7923


0.7923230111072233

In [21]:
def get_unseen_suprise(ratings, df_card, member_no):
    # 특정 유저가 본 card 들을 리스트로 할당
    df_seen_cards = ratings[ratings['member_no'] == member_no]['card_no']
    seen_cards = df_seen_cards.tolist()
    
    # 모든 카드들의 card_no들 리스트로 할당
    total_cards = df_card['card_no'].tolist()
    
    # 모든 카드들 중 특정 유저가 본 카드를 제외한 나머지 추출
    unseen_cards = [card_no for card_no in total_cards if card_no not in seen_cards]
    print(f'특정 {member_no}번 유저가 본 카드 수: {len(seen_cards)}')
    print()
    print(df_card[df_card['card_no'].isin(seen_cards)])

    print()
    print(f'추천한 카드 개수: {len(unseen_cards)}')
    print(f'전체 카드수: {len(total_cards)}')
    
    return unseen_cards

In [22]:
def recomm_card_by_suprise(algo, member_no, unseen_cards, top_n=10):
    #알고리즘 객체의 predict()를 이용해 특정 userId의 평점이 없는 영화들에 대해 평점 예측
    predictions = [algo.predict(str(member_no), str(card_no)) for card_no in unseen_cards]
    
    #predictions는 Prediction()으로 하나의 객체로 되어있기 때문에 예측평점(est값)을 기준으로 정렬해야함
    #est값을 반환하는 함수부터 정의. 이것을 이용해 리스트로 정렬하는 sort()인자의 key값에 넣어주자
    
    def sortkey_est(pred):
        return pred.est
    
    #sortkey_est함수로 리스트를 정렬하는 sort함수의 key인자에 넣어주자
    #리스트 sort는 디폴트값이 inplace=True인 것처럼 정렬되어 나온다. reverse=True가 내림차순
    predictions.sort(key=sortkey_est, reverse=True)
    
    # 공지사항 필터링 : 학습데이터에서 빼버릴까..
    
    
    #상위 n개의 예측값들만 할당
    top_predictions = predictions[:top_n]
    
    # top_predictions에서 card_no, score, card title 뽑아내기
    top_card_no = [int(pred.iid) for pred in top_predictions]
    top_card_score = [pred.est for pred in top_predictions]
    top_card_titles = df_card[df_card.card_no.isin(top_card_no)]['title']
    
    # 위 3가지를 튜플로 담기
    # zip함수를 사용해서 각 자료구조(여기선 리스트)의 똑같은 위치에 있는 값들을 mapping
    # zip함수는 참고로 여러개의 문자열의 똑같은 위치들끼리 mapping도 가능
    top_card_preds = [(card_no, score, title) for card_no, score, title in zip(top_card_no, top_card_score, top_card_titles)] 
    
    return top_card_preds

In [23]:
### 위에서 정의한 함수를 사용해 특정 유저의 추천 영화들 출력
unseen_lst = get_unseen_suprise(ratings, df_card, 106150767)
top_card_preds = recomm_card_by_suprise(algo, 106150767, unseen_lst, 100)

특정 106150767번 유저가 본 카드 수: 12

        card_no wrt_member_no                         title  community_no
11871   6831965      80535993              랑디 세리스 둘중 뭘뽑아야됨?        1005.0
26934   6854835      80026063    불디카 나세실 세리스가 같은 용도로 쓰는건가요?        1005.0
40753   6762964      71168219          오토마톤 타워 클리어 & 공략 이벤트        1000.0
48929   5561180     124988034      보랏 심키세 또미 주슈리조합으로 아레나돌릴때        1186.0
71300   6841560      79545943                        부옵션 뭐고        1005.0
73030   6868529     106036269                           인빛을        1005.0
89675   5841025     124916036     입문자를 위한 아레나 공략본~ 뉴비~에린이까지        1189.0
110115  6871482      71168219      카일론 & 알렉사의 바구니 소환 확률 UP!        1000.0
119720  4823512      70192816                 인도하는빛 아티 질문드림        1005.0
128491  6708852     124916036  뉴비~에린이탈출 까지 아레나 공덱(속덱) 요약본!!        1005.0
283859  6841257     139311016   원정대에 칠수있는 사람수만 제한 풀면 될꺼 같은데        1005.0
319483  6861868      87405218                       인빛? 시추?        1005.0

추천한 카드 

In [28]:
pd.set_option('display.max_rows', 10)
print()
print('#'*8, 'Top-10 추천 카드 리스트', '#'*8)

# top_card_preds가 여러가지의 튜플을 담고 있는 리스트이기 때문에 반복문 수행
for top_card in top_card_preds :
    # df 생성
    df_recommend = df_recommend.append(df_card[df_card['card_no'] == top_card[0]])
    df_recommend['predict'] = top_card[1]
    #print('* 추천 카드 title: ', top_card[2])
    #print('* 해당 카드의 예측평점: ', top_card[1])
    #print()

df_recommend.head(10)


######## Top-10 추천 카드 리스트 ########


,card_no,wrt_member_no,title,community_no,predict
9,1660627,354222,ㅎㅇ,1005.0,1.474166
0,1455534,71168219,8/30(목) 모험 진행 불가 오류 조치 안내,995.0,1.474166
1,1480174,10638853,유저들 가챠로 부자된 알카서스2,1007.0,1.474166
2,1494649,70395475,ㅎㅇ,1006.0,1.474166
3,1499690,10215176,결제한거 안들어 오네요,1012.0,1.474166
4,1514597,79976493,과금에 대처하는 방법 카톡문의 mg074,1005.0,1.474166
5,1525479,79221914,클로에 버프 해주세요,1013.0,1.474166
6,1606009,80140469,추석이벤트 지급누락,1012.0,1.474166
7,1606727,80699566,버그(?),1012.0,1.474166
8,1655583,81325891,이번유피네 좀 그런데,1005.0,1.474166
